In [1]:
# Reproduce the simply synthetic forecast from the Java Neural Network using the sklearn.neural_network.
# https://scikit-learn.org/stable/modules/neural_networks_supervised.html
# 2021-09-04

# *********************************************************
# NOTE: This is deprecated.  See simulatedFootball instead.
# *********************************************************

In [1]:
import pandas as pd

In [1]:
# https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html
#
# Some notes:
#
# random_stateint, RandomState instance, default=None
# Determines random number generation for weights and bias initialization, train-test split if early stopping
# is used, and batch sampling when solver=’sgd’ or ‘adam’. Pass an int for reproducible results across multiple
# function calls. See Glossary.
#
# activation : {‘identity’, ‘logistic’, ‘tanh’, ‘relu’}, default=’relu’
# Activation function for the hidden layer.
#    ‘identity’, no-op activation, useful to implement linear bottleneck, returns f(x) = x
#    ‘logistic’, the logistic sigmoid function, returns f(x) = 1 / (1 + exp(-x)).
#    ‘tanh’, the hyperbolic tan function, returns f(x) = tanh(x).
#    ‘relu’, the rectified linear unit function, returns f(x) = max(0, x)
#
# verbose : bool, default=False
# Whether to print progress messages to stdout.

In [2]:
import numpy as np

In [354]:
# Create training and validation data.
import random
import numpy.random
from numpy.random import default_rng
dataSetSize = 2500
numBins = 20
binSize = 1./numBins
randomSeed = 20210905
random.seed(randomSeed)
default_rng(randomSeed)
allowContinuousOutputValues = True

In [ ]:
# Copying this to simulatedSeason:

# What does a typical season of NCAA Div 1 look like?
#
# 2017 colI_2017wk00out totals: all  16   conf   0   nonConf  14   nonDiv   2
# 2017 colI_2017wk01out totals: all 216   conf   8   nonConf 116   nonDiv  92
# 2017 colI_2017wk02out totals: all 218   conf  24   nonConf 144   nonDiv  50
# 2017 colI_2017wk03out totals: all 220   conf  34   nonConf 158   nonDiv  28
# 2017 colI_2017wk04out totals: all 218   conf 142   nonConf  70   nonDiv   6
# 2017 colI_2017wk05out totals: all 214   conf 166   nonConf  46   nonDiv   2
# 2017 colI_2017wk06out totals: all 222   conf 194   nonConf  26   nonDiv   2
# 2017 colI_2017wk07out totals: all 228   conf 212   nonConf  16   nonDiv   0
# 2017 colI_2017wk08out totals: all 220   conf 206   nonConf  14   nonDiv   0
# 2017 colI_2017wk09out totals: all 226   conf 216   nonConf   8   nonDiv   2
# 2017 colI_2017wk10out totals: all 242   conf 222   nonConf  20   nonDiv   0
# 2017 colI_2017wk11out totals: all 236   conf 222   nonConf  12   nonDiv   2
# 2017 colI_2017wk12out totals: all 240   conf 214   nonConf  16   nonDiv  10
# 2017 colI_2017wk13out totals: all 144   conf 116   nonConf  28   nonDiv   0
# 2017 colI_2017wk14out totals: all  50   conf  38   nonConf  12   nonDiv   0
# 2017 colI_2017wk15out totals: all  10   conf   0   nonConf  10   nonDiv   0
# 2017 colI_2017wk16out totals: all  82   conf   0   nonConf  82   nonDiv   0
# 2017 colI_2017wk17out totals: all   4   conf   2   nonConf   2   nonDiv   0
#
# defAve, offAve - average for that divsion
# defRms, offRms - variation of the def & off inside that division
# defChange, offChange - RMS change for off & def between years
#
# Division IA  defAve  5.43 offAve  7.88 defRms 0.40 offRms 0.48 defChange 0.46 offChange 0.57
# Division IAA defAve  4.88 offAve  6.97 defRms 0.41 offRms 0.48 defChange 0.47 offChange 0.54

In [8]:
# Copying this to simulatedSeason:

class Team:
    nextTeamId = 0
    
    def __init__(self, teamName):
        self.name = teamName
        self.teamId = Team.nextTeamId
        Team.nextTeamId += 1
        self.conference = {}
        self.division = {}
    
    def set_conference(self, year, conference, division):
        self.conference[year] = conference
        self.division[year] = division

print(Team.nextTeamId)
sampleTeam = Team('BYU')
sampleTeam.set_conference(2021, "Independent", "IA")
print(Team.nextTeamId)
print(sampleTeam.conference[2021])

0
1
Independent


In [355]:
# Create the training and validation arrays, filling in with random values which will then be scaled/adjusted.
#X_train = np.empty(dataSetSize,dtype=float)
#X_valid = np.empty(dataSetSize,dtype=float)
#y_train = np.empty([dataSetSize,numBins],dtype=float)
#y_valid = np.empty([dataSetSize,numBins],dtype=float)
X_train = default_rng().random((dataSetSize,1))
X_valid = default_rng().random((dataSetSize,1))
result_train = default_rng().random((dataSetSize))
result_valid = default_rng().random((dataSetSize))
y_train = np.empty((dataSetSize,numBins))
y_valid = np.empty((dataSetSize,numBins))
y_probs = np.arange(start=0.0, stop=1.0, step=binSize)
#print (y_probs)

In [356]:
# Scale X data so that the forecast is for the favored team (i.e. preditions range from 0.5 to 1)
X_train = 0.5*X_train + 0.5
X_valid = 0.5*X_valid + 0.5
# Scale the data for tanh.  Is this needed?
#X_train = 2.*(X_train - 0.5)
#X_valid = 2.*(X_valid - 0.5)
#result_train = 2.*(result_train - 0.5)
#result_valid = 2.*(result_train - 0.5)

In [357]:
# Finish setting the result, which is expected+pertubation.
result_train = result_train + X_train[:,0] - 0.5
result_valid = result_valid + X_valid[:,0] - 0.5

In [358]:
nonBinaryLabelsAllowed = allowContinuousOutputValues # Multioutput target data is not supported with label binarization

# Fill in y according to result > 0
for index in range(dataSetSize):
    for pIndex in range(numBins):
        if (result_train[index] >= y_probs[pIndex]):
            y_train[index,pIndex] = 1.0
        else:
            #y_train[index,pIndex] = -1.0
            y_train[index,pIndex] = 0.0
        if (result_valid[index] >= y_probs[pIndex]):
            y_valid[index,pIndex] = 1.0
        else:
            #y_valid[index,pIndex] = -1.0
            y_valid[index,pIndex] = 0.0
    # Set the value at the transition point
    if (nonBinaryLabelsAllowed):
        iResid = int(result_train[index]/binSize)
        if (iResid >= 0 and iResid < numBins):
            y_train[index,iResid] = result_train[index]/binSize - iResid
        iResid = int(result_valid[index]/binSize)
        if (iResid >= 0 and iResid < numBins):
            y_valid[index,iResid] = result_valid[index]/binSize - iResid
print(dataSetSize)        
print(result_train[0:10])
print(y_train[0:10,:])

2500
[0.13792537 1.13022876 0.56389748 0.66494079 0.45610057 0.78953954
 0.81884201 0.8161164  0.53632294 0.99445029]
[[1.         1.         0.75850738 0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [1.         1.         1.         1.         1.         1.
  1.         1.         1.         1.         1.         1.
  1.         1.         1.         1.         1.         1.
  1.         1.        ]
 [1.         1.         1.         1.         1.         1.
  1.         1.         1.         1.         1.         0.27794963
  0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [1.         1.         1.         1.         1.         1.
  1.         1.         1.         1.         1.         1.
  1.         0.29881577 0.         0.         0.         0.
  0.         0.        ]
 [1.         1.         1.         1.         1.      

In [359]:
# Create and train the model.
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor
if (allowContinuousOutputValues):
    mlp = MLPRegressor(hidden_layer_sizes=(numBins,numBins),activation='tanh',verbose=True,max_iter=5000)
else:
    mlp = MLPClassifier(hidden_layer_sizes=(numBins,numBins),activation='tanh',verbose=True,max_iter=5000)
mlp.fit(X_train,y_train)

Iteration 1, loss = 0.41324839
Iteration 2, loss = 0.29338288
Iteration 3, loss = 0.21265713
Iteration 4, loss = 0.15494356
Iteration 5, loss = 0.11422268
Iteration 6, loss = 0.08878514
Iteration 7, loss = 0.07601225
Iteration 8, loss = 0.07113037
Iteration 9, loss = 0.06950419
Iteration 10, loss = 0.06890478
Iteration 11, loss = 0.06859408
Iteration 12, loss = 0.06844237
Iteration 13, loss = 0.06820568
Iteration 14, loss = 0.06803529
Iteration 15, loss = 0.06789880
Iteration 16, loss = 0.06773212
Iteration 17, loss = 0.06758058
Iteration 18, loss = 0.06741342
Iteration 19, loss = 0.06728463
Iteration 20, loss = 0.06712842
Iteration 21, loss = 0.06701858
Iteration 22, loss = 0.06683567
Iteration 23, loss = 0.06674987
Iteration 24, loss = 0.06658949
Iteration 25, loss = 0.06647015
Iteration 26, loss = 0.06638356
Iteration 27, loss = 0.06626365
Iteration 28, loss = 0.06613180
Iteration 29, loss = 0.06605246
Iteration 30, loss = 0.06591866
Iteration 31, loss = 0.06586813
Iteration 32, los

MLPRegressor(activation='tanh', hidden_layer_sizes=(20, 20), max_iter=5000,
             verbose=True)

In [360]:
# See how well the training data matches.
predictions_train = mlp.predict(X_train)
print("input:")
print(X_train[0:50,0])
print("pred:")
print(predictions_train[0:50,:])
print("train:")
print(y_train[0:50,:])

input:
[0.59699911 0.99916623 0.61723723 0.94726927 0.79059156 0.81991023
 0.97335378 0.58718801 0.51123589 0.55500898 0.74489005 0.7623809
 0.7644073  0.61709111 0.92472435 0.73925784 0.77188456 0.77062802
 0.99062909 0.78333335 0.92786468 0.60331027 0.58971355 0.6151738
 0.90668706 0.57310792 0.82353137 0.55538432 0.82337144 0.50439528
 0.68316438 0.62029158 0.58275025 0.9706761  0.96791077 0.99050878
 0.74675999 0.93762277 0.55794857 0.9711452  0.96739598 0.75800253
 0.88160878 0.7919184  0.67467566 0.5745779  0.51181457 0.82494955
 0.80999431 0.94172392]
pred:
[[0.96061423 0.9645793  0.94028123 0.92725219 0.86790464 0.84321862
  0.78760538 0.73503064 0.6776007  0.6195251  0.59492387 0.56472848
  0.48229037 0.44004305 0.3876153  0.33502093 0.27988697 0.22237746
  0.17517726 0.1232447 ]
 [1.04244961 1.03030844 1.0392549  1.01992736 1.05565054 1.06228201
  1.06046946 1.0345783  1.01623269 1.0390711  0.93249842 0.86169738
  0.85362007 0.83012599 0.77274864 0.7604356  0.71548457 0.65526

In [361]:
# Method to get the resultant probability from a probability confidence array.

def get_probability(prob_array):
    binSize = 1./prob_array.size
    iProb = prob_array.size - 1
    while prob_array[iProb] < 0.5 and iProb > 0:
        iProb -= 1
    prob = binSize*(iProb+prob_array[iProb])
    return prob

In [362]:
# Method to compute skill.
def compute_skill(predicted,actual):
    skill = 0.0
    numBins = predicted[0,:].size
    numSets = predicted[:,0].size
    binSize = 1./numBins
    goodExpectedTotal = np.zeros((3,numBins))
    for index in range(numSets):
        predictedProb = get_probability(predicted[index,:])
        actualProb = get_probability(actual[index,:])
        iBin = int(predictedProb/binSize+0.5)
        if (iBin > numBins - 1):
            iBin = numBins - 1
        if ((predictedProb > 0.5 and actualProb > 0.5) or (predictedProb < 0.5 and actualProb < 0.5) or (predictedProb == 0.5 and actualProb == 0.5)):
            goodExpectedTotal[0,iBin] += 1
        goodExpectedTotal[1,iBin] += predictedProb
        goodExpectedTotal[2,iBin] += 1
    return goodExpectedTotal

In [363]:
# Print skill histogram.
def print_histogram(goodExpectedTotal):
    numBins = goodExpectedTotal[0,:].size
    dProb = 1./numBins
    headerString   = "                 "
    totalString    = "total games:     "
    rightString    = "number right:    "
    expectedString = "number expected: "
    rightExpString = "right/expected:  "
    totalGames = 0
    totalGood = 0
    totalExpected = 0
    for iBin in range(int(0.5*numBins-1.5),numBins):
        iBegPct = int(100*iBin*dProb)
        iEndPct = iBegPct + int(100*dProb-1)
        headerString += ' {0:02d}'.format(iBegPct)
        headerString += '-{0:02d}%'.format(iEndPct)
        totalString += ' {:6d}'.format(int(goodExpectedTotal[2,iBin]+0.5))
        rightString += ' {:6d}'.format(int(goodExpectedTotal[0,iBin]+0.5))
        expectedString += ' {:6.1f}'.format(goodExpectedTotal[1,iBin])
        if (goodExpectedTotal[1,iBin] > 0.0):
            rightExpString += ' {:6.2f}'.format(goodExpectedTotal[0,iBin]/goodExpectedTotal[1,iBin])
        else:
            rightExpString += ' {:6.2f}'.format(0.0)
        totalGames += goodExpectedTotal[2,iBin]
        totalGood += goodExpectedTotal[0,iBin]
        totalExpected += goodExpectedTotal[1,iBin]
    headerString += "    all"
    totalString += ' {:6d}'.format(int(totalGames+0.5))
    rightString += ' {:6d}'.format(int(totalGood+0.5))
    expectedString += ' {:6.1f}'.format(totalExpected)
    if (totalExpected == 0.0):
        totalExpected = 1.0;
    rightExpString += ' {:6.2f}'.format(totalGood/totalExpected)
    print(headerString)
    print(totalString)
    print(rightString)
    print(expectedString)
    print(rightExpString)

In [364]:
trainGoodExpectedTotal = compute_skill(predictions_train,y_train)
print("Training data:")
print_histogram(trainGoodExpectedTotal)

Training data:
                  40-44% 45-49% 50-54% 55-59% 60-64% 65-69% 70-74% 75-79% 80-84% 85-89% 90-94% 95-99%    all
total games:           0      0      0     82    473    202    210    219    301    210    320    483   2500
number right:          0      0      0     44    252    116    124    156    234    175    272    449   1822
number expected:     0.0    0.0    0.0   43.1  272.9  126.5  142.0  159.0  233.7  173.5  280.5  456.7 1887.9
right/expected:     0.00   0.00   0.00   1.02   0.92   0.92   0.87   0.98   1.00   1.01   0.97   0.98   0.97


In [365]:
# See how well the validation data matches.
predictions_valid = mlp.predict(X_valid)
validGoodExpectedTotal = compute_skill(predictions_valid,y_valid)
print("Validation data:")
print_histogram(validGoodExpectedTotal)

Validation data:
                  40-44% 45-49% 50-54% 55-59% 60-64% 65-69% 70-74% 75-79% 80-84% 85-89% 90-94% 95-99%    all
total games:           0      0      0    117    436    206    223    233    280    230    322    453   2500
number right:          0      0      0     62    239    123    145    169    207    180    281    423   1829
number expected:     0.0    0.0    0.0   61.5  251.6  129.0  150.8  169.2  217.4  190.0  282.3  428.6 1880.4
right/expected:     0.00   0.00   0.00   1.01   0.95   0.95   0.96   1.00   0.95   0.95   1.00   0.99   0.97
